In [ ]:
# ==========================================
# INSTALL DEPENDENCIES
# ==========================================
!pip install transformers datasets accelerate

import json
import torch
from transformers import (
    GPT2Tokenizer, 
    GPT2LMHeadModel, 
    Trainer, 
    TrainingArguments, 
    DataCollatorForLanguageModeling
)
from datasets import Dataset

# ==========================================
# CONFIGURATION
# ==========================================

# 1. Path to your dataset (The output from Step 2/3)
DATASET_PATH = "/kaggle/input/tanmay-soni2/unique_dataset (1)_filtered_threshold_0.5.jsonl" 

# 2. Student Model (We use GPT-2 Small as the student, as per the paper)
MODEL_CHECKPOINT = "gpt2" 

# 3. Output directory for the trained model
OUTPUT_DIR = "/kaggle/working/my_comet_student"

# ==========================================
# DATA PREPARATION
# ==========================================

def format_comet_input(head, relation, tail):
    """
    Formats data into the COMET input string:
    <head> Event </head> <relation> Relation </relation> [GEN] Tail
    """
    return f"<head> {head} </head> <relation> {relation} </relation> [GEN] {tail}"

print("Loading and formatting dataset...")
data = []
with open(DATASET_PATH, 'r') as f:
    for line in f:
        try:
            d = json.loads(line)
            # Use 'inference' (from your unique_dataset) or 'tail'
            tail_text = d.get('inference', d.get('tail', ''))
            
            formatted_text = format_comet_input(d['head'], d['relation'], tail_text)
            data.append({"text": formatted_text})
        except:
            continue

# Convert to Hugging Face Dataset
dataset = Dataset.from_list(data)
# Split into Train/Val (90% train, 10% val)
dataset = dataset.train_test_split(test_size=0.1)

print(f"Training on {len(dataset['train'])} examples, Validating on {len(dataset['test'])}")

# ==========================================
# TOKENIZATION
# ==========================================

print("Loading Tokenizer...")
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_CHECKPOINT)

# GPT-2 doesn't have a pad token, so we add one
tokenizer.pad_token = tokenizer.eos_token

# Add Special Tokens (Crucial for COMET)
special_tokens_dict = {
    'additional_special_tokens': ['<head>', '</head>', '<relation>', '</relation>', '[GEN]']
}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=64)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# ==========================================
# TRAINING
# ==========================================

print("Loading Model...")
model = GPT2LMHeadModel.from_pretrained(MODEL_CHECKPOINT)

# Resize embeddings because we added special tokens
model.resize_token_embeddings(len(tokenizer))

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    # REPLACE 'evaluation_strategy' WITH 'eval_strategy'
    eval_strategy="epoch",  
    learning_rate=5e-5,
    weight_decay=0.01,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    save_strategy="epoch",
    save_total_limit=2,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

print("Starting Training...")
trainer.train()

# ==========================================
# SAVING
# ==========================================

print(f"Saving model to {OUTPUT_DIR}...")
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print("Done! You can now use this model for inference.")

In [ ]:
import shutil
import os

# 1. Define the folder you want to download (from your config)
folder_path = "/kaggle/working/my_comet_student"

# 2. Define the output zip filename
zip_name = "/kaggle/working/student_model2"

# 3. Create the zip file
print(f"Zipping {folder_path}...")
shutil.make_archive(zip_name, 'zip', folder_path)

print(f"Success! Created {zip_name}.zip")
print("Check the 'Output' section on the right sidebar to download it.")

In [1]:
from IPython.display import FileLink
import os

# 1. Define the file path (based on your previous step)
zip_file_path = "student_model2.zip"

# 2. Check if it exists
if os.path.exists(zip_file_path):
    print(f"File found: {zip_file_path}")
    print("Click the link below to download:")
    
    # 3. Generate the link
    display(FileLink(zip_file_path))
else:
    print(f"Error: Could not find {zip_file_path}")
    print("Did you run the 'shutil.make_archive' cell above?")
    print("Current files in directory:", os.listdir("."))

Error: Could not find student_model2.zip
Did you run the 'shutil.make_archive' cell above?
Current files in directory: ['.virtual_documents']


In [8]:
# ==========================================
# 1. SETUP & INSTALLATION
# ==========================================
# Run this cell to install dependencies if not already present
!pip install -q transformers torch

import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# ==========================================
# 2. CONFIGURATION
# ==========================================

# Path to your trained model folder
# (Use the one you confirmed works in your previous turns)
MODEL_PATH = "/kaggle/input/student-model2/pytorch/default/1"

# Detect GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# ==========================================
# 3. LOAD MODEL & RECONSTRUCT TOKENIZER
# ==========================================

print(f"Loading model weights from {MODEL_PATH}...")
try:
    # Load the Model Weights
    model = GPT2LMHeadModel.from_pretrained(MODEL_PATH).to(device)
    model.eval()

    print("Reconstructing Tokenizer from base 'gpt2'...")
    # Load base GPT-2 tokenizer (since your folder lacked vocab.json)
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    tokenizer.pad_token = tokenizer.eos_token

    # CRITICAL: Re-add special tokens in the EXACT order used during training
    # This aligns the tokenizer's vocabulary with your trained model
    special_tokens_dict = {
        'additional_special_tokens': ['<head>', '</head>', '<relation>', '</relation>', '[GEN]']
    }
    tokenizer.add_special_tokens(special_tokens_dict)
    
    print("✅ Model and Tokenizer loaded successfully!")

except Exception as e:
    print(f"❌ Error loading model: {e}")
    # Stop execution if model fails to load
    raise e

# ==========================================
# 4. INTERACTIVE CHATBOT FUNCTION
# ==========================================
def run_full_report_chat():
    print("\n" + "="*60)
    print("🤖 COMET REPORT GENERATOR")
    print("="*60)
    print("Type an Event, and I will generate ALL social inferences for it.")
    
    relations_to_check = [
        ('xIntent', 'Why it happened'),
        ('xEffect', 'What happens next'),
        ('xReact',  'How they feel'),
        ('xAttr',   'Personality trait'),
        ('xWant',   'What they want next'),
        ('xNeed',   'What PersonX needed to do before'),
        ('HinderedBy', 'What could prevent this')
    ]

    while True:
        head = input("\n📝 Enter Event (or 'exit'): ").strip()
        if head.lower() in ['exit', 'quit']: break
        
        print(f"\n--- Analysis for: '{head}' ---")
        
        for rel_code, rel_desc in relations_to_check:
            # Prepare input
            input_text = f"<head> {head} </head> <relation> {rel_code} </relation> [GEN]"
            inputs = tokenizer(input_text, return_tensors="pt").to(device)
            
            # Generate
            with torch.no_grad():
                outputs = model.generate(
                    **inputs, 
                   max_new_tokens=20,        # FORCE it to be short (Atomic is rarely >10 words)
    min_length=2,             # Prevent empty answers
    num_beams=5,              # Higher beams = smarter search
    no_repeat_ngram_size=2,   # KILL repetition (e.g. "immature and immature")
    early_stopping=True,      # Stop exactly when the EOS token is found
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id, # Explicitly tell it what "Stop" looks like
    repetition_penalty=1.5    # Strong penalty for repeating phrases
                )
            
            # Decode
            full_text = tokenizer.decode(outputs[0], skip_special_tokens=False)
            inference = full_text.split('[GEN]')[1].split('<')[0].strip()
            
            # Print cleanly
            print(f"🔹 {rel_desc:<20} ({rel_code}): {inference}")

def run_comet_chat():
    print("\n" + "="*60)
    print("🤖 COMET STUDENT MODEL - INTERACTIVE DEMO")
    print("="*60)
    print("Instructions:")
    print("1. Type an Event (e.g., 'PersonX asks for a refund')")
    print("2. Choose a Relation from the list below.")
    print("3. Type 'exit' or 'quit' at any time to stop.")
    print("-" * 60)
    
    valid_relations = [
        'xWant',      # What PersonX wants to do after
        'xEffect',    # Effect on PersonX
        'xReact',     # How PersonX feels
        'xAttr',      # How PersonX is seen (attributes)
        'xNeed',      # What PersonX needed to do before
        'xIntent',    # Why PersonX did this
        'HinderedBy'  # What could prevent this
    ]

    while True:
        # --- INPUT EVENT ---
        print("\n" + "-"*30)
        head = input("📝 Enter Event: ").strip()
        
        if head.lower() in ['exit', 'quit']:
            print("Goodbye! 👋")
            break
        if not head:
            continue

        # --- INPUT RELATION ---
        print(f"   Available Relations: {', '.join(valid_relations)}")
        relation = input("🔗 Enter Relation: ").strip()
        
        if relation.lower() in ['exit', 'quit']:
            print("Goodbye! 👋")
            break

        # --- GENERATION ---
        try:
            # Format the input exactly as the model was trained
            input_text = f"<head> {head} </head> <relation> {relation} </relation> [GEN]"
            inputs = tokenizer(input_text, return_tensors="pt").to(device)
            
            # Generate Inference
            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=24,       # Short, crisp inferences
                    num_beams=5,             # Beam search for quality
                    early_stopping=True,
                    no_repeat_ngram_size=2,
                    pad_token_id=tokenizer.eos_token_id
                )
            
            # Decode output
            full_text = tokenizer.decode(outputs[0], skip_special_tokens=False)
            
            # Parse the result (Extract text after [GEN])
            try:
                inference = full_text.split('[GEN]')[1].strip()
                # Clean up specific end tokens if they appear
                inference = inference.replace('<|endoftext|>', '').strip()
            except IndexError:
                inference = full_text # Fallback if structure is malformed

            # Output
            print("="*60)
            print(f"🧠 Inference:  {inference}")
            print("="*60)

        except Exception as e:
            print(f"❌ Error during generation: {e}")

# ==========================================
# 5. START THE CHAT
# ==========================================
if __name__ == "__main__":
    run_full_report_chat()

Using device: cuda
Loading model weights from /kaggle/input/student-model2/pytorch/default/1...
Reconstructing Tokenizer from base 'gpt2'...
✅ Model and Tokenizer loaded successfully!

🤖 COMET REPORT GENERATOR
Type an Event, and I will generate ALL social inferences for it.



📝 Enter Event (or 'exit'):  john tries to get a new car



--- Analysis for: 'john tries to get a new car' ---
🔹 Why it happened      (xIntent): to be able to afford a car for himself and his family. PersonX wants to drive the car
🔹 What happens next    (xEffect): learns to drive a car and is able to afford it, even though he has no money to buy
🔹 How they feel        (xReact): excited about the car and wants to drive it for a while longer than he has to go to school
🔹 Personality trait    (xAttr): confident in his car skills and wants to buy a car for himself. PersonX is not interested in
🔹 What they want next  (xWant): to buy a car for himself and drive it for a friend who is willing to help him out with
🔹 What PersonX needed to do before (xNeed): to buy a car and drive it well enough to be able to afford it. PersonX can't
🔹 What could prevent this (HinderedBy): PersonX can't find a car dealer's listing for the car he wants to buy for himself.


KeyboardInterrupt: Interrupted by user

In [ ]:
# ==========================================
# 1. INSTALL METRICS
# ==========================================
!pip install evaluate rouge_score nltk

import json
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import evaluate
from tqdm import tqdm

# ==========================================
# 2. CONFIGURATION
# ==========================================

# Path to your TRAINED student model (output of the previous step)
MODEL_PATH = "/kaggle/input/student-model2/pytorch/default/1"

# Path to your data
DATA_PATH = "/kaggle/input/tanmay-soni2/unique_dataset (1)_filtered_threshold_0.5.jsonl"

# Use GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"

# ==========================================
# 3. LOAD MODEL & RECONSTRUCT TOKENIZER
# ==========================================

print(f"Loading model weights from {MODEL_PATH}...")
# 1. Load the Model Weights (This works because the model file is there)
model = GPT2LMHeadModel.from_pretrained(MODEL_PATH).to(device)
model.eval()

print("Reconstructing Tokenizer from base 'gpt2'...")
# 2. Load the base GPT-2 tokenizer (downloading fresh from Hugging Face)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# 3. RE-ADD SPECIAL TOKENS
# CRITICAL: These must be added in the EXACT same order as your training script
# so that they get the same IDs (indices) that the model learned.
special_tokens_dict = {
    'additional_special_tokens': ['<head>', '</head>', '<relation>', '</relation>', '[GEN]']
}
tokenizer.add_special_tokens(special_tokens_dict)

print("Tokenizer reconstructed and synchronized.")

print("Loading Test Data...")
# We need to group references: (Head + Relation) -> [List of valid inferences]
test_data = {}

with open(DATA_PATH, 'r') as f:
    for line in f:
        try:
            d = json.loads(line)
            if d.get('split') == 'test':
                key = (d['head'], d['relation'])
                inference = d.get('inference', d.get('tail', ''))
                
                if key not in test_data:
                    test_data[key] = []
                test_data[key].append(inference)
        except:
            continue

print(f"Found {len(test_data)} unique test prompts.")
# ==========================================
# 4. GENERATION FUNCTION
# ==========================================

def generate_inference(head, relation):
    # Format input exactly like training: <head> ... <relation> ... [GEN]
    input_text = f"<head> {head} </head> <relation> {relation} </relation> [GEN]"
    
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=24,      # Inferences are usually short
            num_beams=3,            # Beam search for better quality
            early_stopping=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    # Decode and remove the input prompt part
    full_text = tokenizer.decode(outputs[0], skip_special_tokens=False)
    # Extract text after [GEN]
    try:
        generated_text = full_text.split('[GEN]')[1].strip()
        # Remove special tokens if they leaked
        generated_text = generated_text.replace('<|endoftext|>', '').strip()
    except:
        generated_text = full_text # Fallback
        
    return generated_text

# ==========================================
# 5. RUN EVALUATION LOOP
# ==========================================

predictions = []
references = []

print("Generating predictions (This may take a few minutes)...")
# Limit to first 200 for speed if you want a quick check, or remove [:200] for full eval
test_keys = list(test_data.keys()) # [:200] 

for head, relation in tqdm(test_keys):
    # 1. Generate
    pred = generate_inference(head, relation)
    predictions.append(pred)
    
    # 2. Get References (Teacher's answers)
    refs = test_data[(head, relation)]
    references.append(refs)

# ==========================================
# 6. CALCULATE SCORES
# ==========================================

bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

print("\nCalculating Metrics...")

# BLEU
bleu_score = bleu.compute(predictions=predictions, references=references)
print(f"BLEU Score: {bleu_score['bleu']:.4f}")

# ROUGE
rouge_score = rouge.compute(predictions=predictions, references=references)
print(f"ROUGE-L: {rouge_score['rougeL']:.4f}")

# ==========================================
# 7. QUALITATIVE CHECK (SEE EXAMPLES)
# ==========================================

print("\n=== QUALITATIVE EXAMPLES ===")
for i in range(5):
    head, rel = test_keys[i]
    print(f"Event:    {head}")
    print(f"Relation: {rel}")
    print(f"Student:  {predictions[i]}")
    print(f"Teacher:  {test_data[(head, rel)]}")
    print("-" * 40)

In [ ]:
!pip install transformers==4.30.0 torch accelerate bitsandbytes
!pip install ftfy names scipy scikit-learn

In [ ]:
pip install -U bitsandbytes

In [1]:
import json
import torch
import evaluate
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# ==========================================
# 1. LOAD TEACHER (4-Bit Quantized)
# ==========================================
teacher_name = "EleutherAI/gpt-j-6B"

print(f"Loading Teacher: {teacher_name}...")

tokenizer = AutoTokenizer.from_pretrained(teacher_name)
tokenizer.pad_token = tokenizer.eos_token # GPT-J needs a pad token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    teacher_name,
    quantization_config=bnb_config,
    device_map="auto",
)

# ==========================================
# 2. DEFINE PROMPT TEMPLATES
# ==========================================
# The Teacher is a base model, so we prompt it with sentence completion.
# These map the Relation to a natural language phrase.
RELATION_TEMPLATES = {
    'xWant': "{head}, as a result, PersonX wants",
    'xEffect': "{head}, as a result, PersonX",
    'xReact': "{head}, as a result, PersonX feels",
    'xIntent': "{head}, because PersonX wanted",
    'xNeed': "{head}, but before, PersonX needed",
    'xAttr': "{head}, so PersonX is seen as",
    'HinderedBy': "{head}. This can be hindered by",
}

def construct_teacher_prompt(head, relation):
    # Get the template or default to a generic one
    template = RELATION_TEMPLATES.get(relation, "{head} {relation}")
    # Fill in the event
    prompt = template.format(head=head, relation=relation)
    return prompt

# ==========================================
# 3. LOAD TEST DATA
# ==========================================
DATA_PATH = "/kaggle/input/tanmay-soni2/unique_dataset (1)_filtered_threshold_0.5.jsonl" # Your dataset

print("Loading Test Data...")
test_data = {}

# We group references by prompt so we can compare 1 Gen vs All Valid Refs
with open(DATA_PATH, 'r') as f:
    for line in f:
        try:
            d = json.loads(line)
            # Only look at the TEST split
            if d.get('split') == 'test':
                # Key = (Head, Relation)
                key = (d['head'], d['relation'])
                inference = d.get('inference', d.get('tail', ''))
                
                if key not in test_data:
                    test_data[key] = []
                test_data[key].append(inference)
        except:
            continue

print(f"Found {len(test_data)} unique test prompts.")

# ==========================================
# 4. EVALUATION LOOP
# ==========================================
predictions = []
references = []

print("Generating Teacher predictions...")
# We limit to 50 for speed demonstration (Remove [:50] for full run)
test_keys = list(test_data.keys())[:50] 

for head, relation in tqdm(test_keys):
    # A. Format Input (Natural Language)
    prompt = construct_teacher_prompt(head, relation)
    
    # B. Tokenize
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    
    # C. Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=15,      # Atomic inferences are short
            do_sample=False,        # Greedy for deterministic eval
            pad_token_id=tokenizer.eos_token_id
        )
    
    # D. Decode and Clean
    full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract only the newly generated part (remove the prompt)
    # GPT-J output: "Prompt + Generation"
    generated_text = full_text[len(prompt):].strip()
    
    # Simple cleanup (stop at newline or period if needed)
    generated_text = generated_text.split('\n')[0].strip()
    
    predictions.append(generated_text)
    references.append(test_data[(head, relation)])

# ==========================================
# 5. CALCULATE METRICS
# ==========================================
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

print("\n=== TEACHER RESULTS ===")

# BLEU
bleu_score = bleu.compute(predictions=predictions, references=references)
print(f"BLEU Score: {bleu_score['bleu']:.4f}")

# ROUGE
rouge_score = rouge.compute(predictions=predictions, references=references)
print(f"ROUGE-L: {rouge_score['rougeL']:.4f}")

# ==========================================
# 6. SHOW EXAMPLES
# ==========================================
print("\n=== EXAMPLES ===")
for i in range(3):
    head, rel = test_keys[i]
    print(f"Prompt:   {construct_teacher_prompt(head, rel)}")
    print(f"Teacher:  {predictions[i]}")
    print(f"Refs:     {references[i][:2]}") # Show first 2 refs
    print("-" * 30)

2026-02-03 14:22:44.521728: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770128564.543158     309 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770128564.549687     309 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770128564.567130     309 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770128564.567150     309 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770128564.567153     309 computation_placer.cc:177] computation placer alr

Loading Teacher: EleutherAI/gpt-j-6B...


tokenizer_config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/24.2G [00:00<?, ?B/s]

Some weights of the model checkpoint at EleutherAI/gpt-j-6B were not used when initializing GPTJForCausalLM: ['transformer.h.0.attn.bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.10.attn.bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.11.attn.bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.12.attn.bias', 'transformer.h.12.attn.masked_bias', 'transformer.h.13.attn.bias', 'transformer.h.13.attn.masked_bias', 'transformer.h.14.attn.bias', 'transformer.h.14.attn.masked_bias', 'transformer.h.15.attn.bias', 'transformer.h.15.attn.masked_bias', 'transformer.h.16.attn.bias', 'transformer.h.16.attn.masked_bias', 'transformer.h.17.attn.bias', 'transformer.h.17.attn.masked_bias', 'transformer.h.18.attn.bias', 'transformer.h.18.attn.masked_bias', 'transformer.h.19.attn.bias', 'transformer.h.19.attn.masked_bias', 'transformer.h.2.attn.bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.20.attn.bi

Loading Test Data...
Found 769 unique test prompts.
Generating Teacher predictions...


100%|██████████| 50/50 [00:50<00:00,  1.02s/it]



=== TEACHER RESULTS ===
BLEU Score: 0.0631
ROUGE-L: 0.3197

=== EXAMPLES ===
Prompt:   PersonX says something surprising. This can be hindered by
Teacher:  the fact that the player is not in the same room as the NPC.
Refs:     ['PersonX is afraid to tell his parents about the baby', "PersonX doesn't say anything surprising"]
------------------------------
Prompt:   PersonX says something surprising, but before, PersonX needed
Teacher:  to be a person.
Refs:     ['to be thoughtful', 'to be honest']
------------------------------
Prompt:   PersonX says something surprising, as a result, PersonX wants
Teacher:  to know what PersonY thinks.
Refs:     ['to be a good friend', 'to avoid talking to PersonX']
------------------------------
